In [1]:
import pandas as pd
import numpy as np

In [40]:
movie=pd.read_csv(r'./ml-20m/movies.csv')
tags=pd.read_csv(r'./ml-20m/tags.csv')

In [4]:
movie['genres']=movie['genres'].str.split('|').str.join(',')

In [5]:
movie['title']=movie['title'].str.replace(r'\(\d{4}\)','')
movie['title']=movie['title'].str.rstrip()
movie['title']=movie['title'].str.lstrip()
movie.head()

,movieId,title,genres
0,1,Toy Story,"Adventure,Animation,Children,Comedy,Fantasy"
1,2,Jumanji,"Adventure,Children,Fantasy"
2,3,Grumpier Old Men,"Comedy,Romance"
3,4,Waiting to Exhale,"Comedy,Drama,Romance"
4,5,Father of the Bride Part II,Comedy


In [6]:
tags.drop(['timestamp','userId'],axis=1,inplace=True)

In [7]:
tags.isnull().sum()

movieId     0
tag        16
dtype: int64

In [8]:
tags.fillna('',inplace=True)

In [9]:
keywords=pd.DataFrame(tags.groupby('movieId')['tag'].apply(','.join))
keywords

,tag
movieId,
1,"Watched,computer animation,Disney animated fea..."
2,"time travel,adapted from:book,board game,child..."
3,"old people that is actually funny,sequel fever..."
4,"chick flick,revenge,characters,chick flick,cha..."
5,"Diane Keaton,family,sequel,Steve Martin,weddin..."
...,...
131054,dinosaurs
131082,"documentary,Yoshitomo Nara"
131164,Vietnam War


In [10]:
movie.shape

(27278, 3)

In [11]:
df=pd.merge(movie,keywords,on='movieId',how='left')

In [12]:
df[df['title'].str.contains('Fifty')]

,movieId,title,genres,tag
11632,51019,Fifty-Fifty (a.k.a. Schizo) (Shiza),"Crime,Drama,Romance,Thriller",NaN
13077,62796,Fifty Pills,Comedy,"Kristen Bell,Nora Zehetner"
14306,71691,Fifty Dead Men Walking,"Action,Drama,Thriller","action,crime,double agents,Irish conflict,Jim ..."
26159,125916,Fifty Shades of Grey,Drama,"abuse,abuse,book,erotic,bkk,BDSM,sexual object..."
27166,130836,Fifty/Fifty,"Action,Comedy,Thriller",NaN


In [13]:
df.fillna('',inplace=True)

In [14]:
df['soup']=df['genres']+','+df['tag']
df

,movieId,title,genres,tag,soup
0,1,Toy Story,"Adventure,Animation,Children,Comedy,Fantasy","Watched,computer animation,Disney animated fea...","Adventure,Animation,Children,Comedy,Fantasy,Wa..."
1,2,Jumanji,"Adventure,Children,Fantasy","time travel,adapted from:book,board game,child...","Adventure,Children,Fantasy,time travel,adapted..."
2,3,Grumpier Old Men,"Comedy,Romance","old people that is actually funny,sequel fever...","Comedy,Romance,old people that is actually fun..."
3,4,Waiting to Exhale,"Comedy,Drama,Romance","chick flick,revenge,characters,chick flick,cha...","Comedy,Drama,Romance,chick flick,revenge,chara..."
4,5,Father of the Bride Part II,Comedy,"Diane Keaton,family,sequel,Steve Martin,weddin...","Comedy,Diane Keaton,family,sequel,Steve Martin..."
...,...,...,...,...,...
27273,131254,Kein Bund für's Leben,Comedy,,"Comedy,"
27274,131256,"Feuer, Eis & Dosenbier",Comedy,,"Comedy,"
27275,131258,The Pirates,Adventure,"bandits,Korea,mutiny,pirates,whale","Adventure,bandits,Korea,mutiny,pirates,whale"
27276,131260,Rentun Ruusu,(no genres listed),,"(no genres listed),"


In [15]:
df[df['movieId']==7]

,movieId,title,genres,tag,soup
6,7,Sabrina,"Comedy,Romance","Greg Kinnear,romance,remadeAs:Sabrina(1995),re...","Comedy,Romance,Greg Kinnear,romance,remadeAs:S..."


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tfv=TfidfVectorizer(min_df=3,ngram_range=(1,3),stop_words='english',strip_accents='unicode',analyzer='word',
                   token_pattern=r'\w{1,}')

In [18]:
tfvm=tfv.fit_transform(df['soup'])

In [19]:
tfvm.shape

(27278, 52783)

In [20]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(tfvm,tfvm)

In [22]:
def get_rec(title,sig=sig):
    idx=df.index[df['title']==title][0]
    sigs=list(enumerate(sig[idx]))
    sigs=sorted(sigs,key= lambda x:x[1],reverse=True)
    req=sigs[0:11]
    mov=[i[0] for i in req]
    temp=df['title'].iloc[mov]
    df_out = temp.to_frame()
    output=[x for x in df_out['title']]
    return output

In [24]:
get_rec('Revenant, The')

['Revenant, The',
 'Parents',
 'Villain, The (Le Vilain)',
 'Doghouse',
 'Centenarian Who Climbed Out the Window and Vanished, The (Hundraåringen som klev ut genom fönstret och försvann)',
 'Crimes Against Humanity',
 'Welcome to the Roses (Bienvenue chez les Rozes)',
 'Sexy Nights of the Living Dead',
 'Dark Backward, The',
 'Sightseers',
 'Grabbers']

In [31]:
import pickle

In [36]:
with open('model.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([sig, df], f,)